In [1]:
import os
import random
import pandas as pd
from tqdm import tqdm

In [2]:
!pip install openpyxl
my_random = random.seed(42)
eval_data_folder = "/lpai/volumes/ss-sai-lx-my/zhaojiufeng/tool_script/cua/plan/local_code/evaluation_dataset/"
file_names = ["20250505", "20250504", "20250502"]

Looking in indexes: https://artifactory.ep.chehejia.com/artifactory/api/pypi/pypi-remote/simple

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
def find_badcase(df):
    badcase = []
    for _, row in df.iterrows():
        if 'tool_acc' in row and (row['tool_acc'] == 0 or row['slot_acc'] == 0 or row['query_acc'] == 0):
                badcase.append(row['conversations'])
        if 'acc' in row and (row['acc'] == 0):
            badcase.append(row['conversations'])
    return list(set(badcase))

In [4]:
badcase = []
for name in file_names:
    eval_data_file = os.path.join(eval_data_folder, f"场景规划_自测评估_{name}.xlsx")
    df = pd.read_excel(eval_data_file, sheet_name="easy")
    badcase.extend(find_badcase(df))
print("badcase总数：", len(badcase))

badcase总数： 121


In [5]:
def get_final_round(conversations):
    if pd.isna(conversations):  # 处理缺失
        return ''
    conversations_list = conversations.strip().split('\n')
    if conversations_list and conversations_list[-1].startswith('user'):
        return conversations_list[-2] + '\n' + conversations_list[-1]
    return ''

In [6]:
# file_name = "20250505"
# eval_data_file = os.path.join(eval_data_folder, f"场景规划_自测评估_{file_name}.xlsx")
eval_data_file = "/lpai/volumes/ss-sai-lx-my/zhaojiufeng/tool_script/cua/plan/local_code/evaluation_dataset/场景规划评估集_V1.xlsx"
df = pd.read_excel(eval_data_file, sheet_name="自测集-easy")
print("*****去重前*****")
print("总量：", len(df))
print("start数量：", (df["工具"] == "cu_agent_start").sum())
print("continue数量：", (df["工具"] == "cu_agent_continue").sum())
print("other数量：", (df["工具"] == "other").sum())
print('\n')

*****去重前*****
总量： 5570
start数量： 146
continue数量： 1647
other数量： 3777




In [7]:
def get_coffee_app(row):
    coffee_apps = ["星巴克", "瑞幸"]
    coffee_app = ""
    for app in coffee_apps:
        if app in row["对话"]:
            coffee_app = app
            break
    return coffee_app

In [8]:

# start全部保留
df_start = df[df["工具"] == "cu_agent_start"].copy()

# continue分组
df_continue = df[df["工具"] == "cu_agent_continue"].copy()
df_continue["final_round"] = df_continue["对话"].apply(get_final_round)
grouped = df_continue.groupby('final_round')

df_continue_list = []
for final_round, group in grouped:
    # 判断本组是否全部在badcase
    if group["对话"].isin(badcase).all():
        # 全部都是badcase，全部保留
        df_continue_list.append(group)
    else:
        # 非badcase的分组：如果数量大于20，采样5条，否则全保留
        # group_unique = group.drop_duplicates(subset='conversations', keep='first')
        if len(group) > 20:
            group_sample = group.sample(5, random_state=42)
            df_continue_list.append(group_sample)
        else:
            df_continue_list.append(group)

df_continue_new = pd.concat(df_continue_list, ignore_index=True)

# other随机采样
rows_to_drop = []
df_other = df[df["工具"] == "other"].copy()
for idx, row in df_other.iterrows():
    if row["对话"] in badcase:
        continue
    else:
        random_num = random.choice([1,2,3,4,5,6])
        if random_num != 3:
            rows_to_drop.append(idx)
            
df_other = df_other.drop(index=rows_to_drop).reset_index(drop=True)
df_other.drop_duplicates(subset='对话', inplace=True, keep='first')

# 合并
df_coffee = pd.concat([df_start, df_continue_new, df_other], ignore_index=True)

df_coffee["场景"] = "咖啡"
# df_coffee["小程序"] = df_coffee.apply(
#     lambda row: "瑞幸" if "瑞幸" in row["对话"] else "星巴克", 
#     axis=1
# )
df_coffee["小程序"] = df_coffee.apply(lambda row: get_coffee_app(row), axis=1)
df_coffee = df_coffee[["对话", "工具", "参数", "场景", "小程序"]]
print("*****去重后总数*****")
print("总量:", len(df_coffee))
print("start数量:", (df_coffee["工具"] == "cu_agent_start").sum())
print("continue数量:", (df_coffee["工具"] == "cu_agent_continue").sum())
print("other数量:", (df_coffee["工具"] == "other").sum())

*****去重后总数*****
总量: 1040
start数量: 146
continue数量: 204
other数量: 690


In [9]:
def get_tea_app(row):
    tea_apps = ["蜜雪冰城", "霸王茶姬", "古茗茶饮", "喜茶"]
    tea_app = ""
    for app in tea_apps:
        if app in row["对话"]:
            tea_app = app
            break
    return tea_app

In [10]:
# 场景扩充-奶茶
coffee_product = {
            "星巴克": ["浓缩咖啡", "美式咖啡", "拿铁", "卡布奇诺", "摩卡", "焦糖玛奇朵", "星冰乐", "红茶拿铁", "抹茶拿铁",
                    "水果茶", "热巧克力", "热苹果酒", "茶星冰乐", "果汁星冰乐", "馥芮白"],
            "瑞幸咖啡": ["美式咖啡", "拿铁", "生椰拿铁", "椰云拿铁", "陨石拿铁", "厚乳拿铁", "酱香拿铁"],
        }
tea_product = {
            "蜜雪冰城": ["柠檬水", "蜜桃四季春", "草莓啵啵", "蓝莓摇摇奶昔", "鲜橙益菌多", "棒打西柚", "棒打鲜橙",
                     "柠檬芦荟茶", "芝士红茶", "芝士四季春", "芝士绿茶", "蜂蜜柚子茶", "柠檬绿茶",
                     "红豆奶茶", "四季春奶茶", "三拼霸霸奶茶", "珍珠奶茶"],
            "霸王茶姬": ["悠悠玫瑰", "春日桃桃", "乌山可可", "霸气芝士奶绿", "霸气柠檬绿", "百香果绿茶", "牛油果霸霸茶", "寻香山茶",
                     "浮生梦媞", "伯牙绝弦", "东美人", "夏梦玫珑", "花田乌龙", "七里香", "桃花茉", "白雾红尘", "桂馥兰香", "醒时春山"],
            "古茗茶饮": ["招牌奶茶", "古茗奶茶", "大叔奶茶", "黑糖珍珠奶茶", "冰淇淋奶茶", "桃花醉", "桂花酒酿小丸子",
                     "抹茶绿意", "菊花乌龙", "龙井茶", "果蔬养生茶", "桂花龙眼椰", "桂花乌龙奶芙", "茉香云顶抹茶", "芋见青稞抹茶",
                     "杨枝甘露", "芒果冰沙", "泡鲁达", "芝士椰椰", "西瓜椰椰", "百香双重奏", "超A芝士葡萄"],
            "喜茶": ["多肉葡萄", "芝芝莓莓", "多肉芒芒甘露", "芝芝芒芒", "清爽巴乐提", "清爽巴乐葡", "芝芝桃桃", "满杯红柚",
                   "烤黑糖波波牛乳", "热芋泥波波牛乳", "冰博克超厚牛乳波波", "桂花乌龙", "多肉青提冻", "生打椰椰奶冻"],
        }
all_coffee_products = [item for v in coffee_product.values() for item in v]
df_tea = df_coffee.copy()

for idx, row in df_tea.iterrows():
    conversations = row["对话"].replace("咖啡", "奶茶")
    arguments = row["参数"].replace("咖啡", "奶茶")
    for app, products in coffee_product.items():
        if app in conversations:
            # 随机选择替换的茶饮品牌和产品
            tea_apps = list(tea_product.keys())
            tea_app = random.choice(tea_apps)
            for product in products:
                if product in conversations:
                    tea_items = tea_product[tea_app]
                    tea_item = random.choice(tea_items)
                    # 替换app、product
                    conversations = conversations.replace(app, tea_app).replace(product, tea_item)
                    arguments = arguments.replace(app, tea_app).replace(product, tea_item)
                    # 只换第一次就跳出循环
            else:
                 conversations = conversations.replace(app, tea_app)
                 arguments = arguments.replace(app, tea_app)
        else:
            tea_apps = list(tea_product.keys())
            tea_app = random.choice(tea_apps)
            tea_items = tea_product[tea_app]
            tea_item = random.choice(tea_items)
            for product in products:
                if product in conversations:
                    conversations = conversations.replace(product, tea_item)
                    arguments = arguments.replace(product, tea_item)
                
            
    df_tea.at[idx, "对话"] = conversations
    df_tea.at[idx, "参数"] = arguments
df_tea.drop_duplicates(subset='对话', inplace=True, keep='first')
df_tea["场景"] = "奶茶"
df_tea["小程序"] = df_tea.apply(lambda row: get_tea_app(row), axis=1)
df_tea = df_tea[["对话", "工具", "参数", "场景", "小程序"]]
print(len(df_tea["对话"]))
print(df_tea.head())

964
                                                  对话              工具  \
0  user: 点杯奶茶\nassistant: 请选择您要使用的小程序\nuser: 喜茶\n...  cu_agent_start   
1  user: 点杯奶茶\nassistant: 请选择您要使用的小程序\nuser: 古茗茶饮...  cu_agent_start   
2  user: 点杯奶茶\nassistant: 请选择您要使用的小程序\nuser: 蜜雪冰城...  cu_agent_start   
3  user: 点杯奶茶\nassistant: 请选择您要使用的小程序\nuser: 蜜雪冰城...  cu_agent_start   
4  user: 点杯奶茶\nassistant: 请选择您要使用的小程序\nuser: 喜茶\n...  cu_agent_start   

                                                  参数  场景   小程序  
0  {\n    "taskType": "点单",\n    "query": "点一个大杯热...  奶茶    喜茶  
1  {\n    "taskType": "点单",\n    "query": "要一个超大杯...  奶茶  古茗茶饮  
2  {\n    "taskType": "点单",\n    "query": "点一个大杯热...  奶茶  蜜雪冰城  
3  {\n    "taskType": "点单",\n    "query": "点一个大杯热...  奶茶  蜜雪冰城  
4  {\n    "taskType": "点单",\n    "query": "要一个超大杯...  奶茶    喜茶  


In [11]:
# 合并奶茶、咖啡场景
df_res = pd.concat([df_coffee, df_tea], ignore_index=True)

In [12]:
# 写入
save_path = os.path.join(eval_data_folder, f"场景规划评估集_V2.xlsx")
df_res.to_excel(save_path, sheet_name="easy", index=True)